## Create Text_Dict dictionary
This file will convert the raw text files for the different text files into a dictinary where each element is {"scene","text"} for example Text_Dict["much_ado_about_nothing_act2scene2"]="Enter Don John and Borach...". This dictinary will later be combined with a dictionary that also contains the summarization of the article.

In [3]:
#Set up
raw_text_location = "../data/raw/Text/"

#Importing raw text into strings
def read_raw_text(file_name):
    with open(raw_text_location+file_name, "r") as f:
        contents = f.read()
    return contents

In [ ]:
#import raw text and store in variabels
hamlet_raw_text = read_raw_text("Hamlet.txt") 
Julius_Caesar_raw_text = read_raw_text("Julius_Caesar.txt")
Macbeth_raw_text = read_raw_text("Macbeth.txt") 
Much_Ado_About_Nothing_raw_text = read_raw_text("Much_Ado_About_Nothing.txt")

Othello_raw_text = read_raw_text("Othello.txt")
Romeo_and_Juliet_raw_text = read_raw_text("Romeo_and_Juliet.txt")
The_Taming_Of_The_Shrew_raw_text = read_raw_text("The_Taming_Of_The_Shrew.txt")
Twelfth_Night_raw_text = read_raw_text("Twelfth_Night.txt")

In [8]:
#Helper Functions

In [262]:
#takes in a string of text (a Shakespeor from Guttentberg) and returns a dictionary with {scene_name: text}
#For example return_var["much_ado_about_nothing_act2scene2"]="Enter Don John and Borach..."]
#Input must be in traditional format of including scene names at the top in a table of contents
def scene_breakout_scenes(raw_text_file, story_name):
    
    #stores scene name in list with each element tuple of structure 
    # ("Act III", "Scene IV.", "Another room in the Castle")
    contents_location=raw_text_file.find("Contents")

    #remove text prior to contents
    raw_text_file=raw_text_file[contents_location:]

    current_act="ACT 0"
    current_scene="SCENE 0."

    scene_name_list=[]

    #loop through contents lines storing scene names in scene_name_list
    #removes each line from raw_text_file after parsed
    while(raw_text_file.find("\n\n\n\n")>0):        
        new_line_loc = raw_text_file.find("\n")
        current_line=raw_text_file[0:new_line_loc]
        raw_text_file=raw_text_file[new_line_loc+1:]

        if current_line[:5]==" ACT ":
            current_act=current_line[1:]
            current_scene=0
            continue

        if current_line[:7]==" Scene ":
            dot_location = current_line.find(".")
            current_scene=current_line[1:dot_location]
            scene_name=current_act+"-"+current_scene+"-"+current_line[dot_location+2:]
            scene_name_list.append(scene_name)
            #scene_name now is a string that looks like ACT II-Scene I-A room in Polonius’s house
    
    scene_start_loc_list=[]

    #poplulate scene_start_loc_list with where each of the scene starts 
    for i in range(len(scene_name_list)):
        first_dash=scene_name_list[i].find("-")
        second_dash=scene_name_list[i][first_dash+1:].find("-")+first_dash+1
        
        #find the next scene in the text
        scene_name = scene_name_list[i][first_dash+1:second_dash].upper()+". "+scene_name_list[i][second_dash+1:]
        if i==0:
            scene_start_loc = raw_text_file.find(scene_name)
        else:
            last_start_loc=scene_start_loc_list[i-1]
            scene_start_loc = raw_text_file[last_start_loc:].find(scene_name)+last_start_loc
        scene_start_loc_list.append(scene_start_loc)

    #add end of book to scene_start_loc_list
    scene_start_loc_list.append(raw_text_file.find("*** END OF THE PROJECT GUTENBERG"))

    return_dict={} #will take form {scene_name: text}
    for i in range(len(scene_name_list)):
        start_loc = scene_start_loc_list[i] #start location of scene inclcuding scene title
        start_loc_no_title = raw_text_file.find("\n",start_loc)

        end_loc = scene_start_loc_list[i+1]
        
        scene_text=raw_text_file[start_loc_no_title:end_loc]
        return_dict[story_name+"-"+scene_name_list[i]] = scene_text

    return return_dict




In [288]:
scene_dict_Romeo_and_Juliet = scene_breakout_scenes(Romeo_and_Juliet_raw_text, "Romeo_and_Juliet") 
print(scene_dict_Romeo_and_Juliet)

{}


In [ ]:
#works with format above
scene_dict_hamlet = scene_breakout_scenes(hamlet_raw_text, "Hamlet") 
scene_dict_Julius_Caesar = scene_breakout_scenes(Julius_Caesar_raw_text, "Julius_Caesar") 
scene_dict_The_Taming_Of_The_Shrew = scene_breakout_scenes(The_Taming_Of_The_Shrew_raw_text, "The_Taming_Of_The_Shrew")
scene_dict_Othello = scene_breakout_scenes(Othello_raw_text, "Othello")


scene_dict_Macbeth = scene_breakout_scenes(Macbeth_raw_text, "Macbeth") #different format
scene_dict_Much_Ado_About_Nothing = scene_breakout_scenes(Much_Ado_About_Nothing_raw_text, "Much_Ado_About_Nothing") #Slightly different
scene_dict_Romeo_and_Juliet = scene_breakout_scenes(Romeo_and_Juliet_raw_text, "Romeo_and_Juliet") #looks the same
scene_dict_Twelfth_Night = scene_breakout_scenes(Twelfth_Night_raw_text, "Twelfth_Night") #looks the same
